In [1]:
import os
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import geopandas as gpd
import rasterio as rio
from shapely.geometry import Polygon

In [14]:
home = str(Path.home())

path_to_forest_bounds = os.path.join(
    home, "Desktop", "drought-tracker", "data", "USDA", "S_USA.AdministrativeRegion.shp")

forests = gpd.read_file(path_to_forest_bounds)

path_to_siskiyou = os.path.join(home, "Desktop", "drought-tracker", "data", "TIGER", "siskiyou.geojson")

siskiyou = gpd.read_file(path_to_siskiyou)

In [19]:
forests.REGIONNAME

0             Northern
1         Southwestern
2        Intermountain
3       Rocky Mountain
4    Pacific Northwest
5    Pacific Southwest
6             Southern
7               Alaska
8              Eastern
Name: REGIONNAME, dtype: object

In [22]:
psw_forests = forests[forests.REGIONNAME == "Pacific Southwest"]

In [23]:
# Convert Siskiyou to EPSG: 4269
siskiyou_4269 = siskiyou.to_crs("EPSG:4269")

C:\Users\dphil\Miniconda3\envs\earth-analytics-python\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [24]:
# Create a single polygon object for clipping
poly = siskiyou_4269.geometry.unary_union
spatial_index = psw_forests.sindex

bbox = poly.bounds

sidx = list(spatial_index.intersection(bbox))

shp_sub = psw_forests.iloc[sidx]

clipped = shp_sub.copy()

clipped = shp_sub.intersection(poly)

final_clipped = clipped[clipped.geometry.notnull()]


In [26]:
final_clipped

5    POLYGON ((-122.67814 42.00462, -122.67814 42.0...
dtype: geometry

In [ ]:
raise ValueError("Test cells below")